In [42]:
#import dependencies
import os
import pandas as pd
import glob

In [45]:
#I want to display the data to better understand what I am working with
#path = 'low_freq/house_1/'
#forward slash to access path and a one more slash to access the folder of the file
#house 4 and 6has air conditionong system
path = 'E:/Document/Master/Dataset/Energy_disaggregation/REDD_dataset/low_freq/house_1/' 
file = path + 'channel_1.dat'
df = pd.read_table(file, sep= ' ')

NameError: name 'house' is not defined

In [44]:
df.head()

,1303132929,224.19
0,1303132930,225.57
1,1303132931,226.09
2,1303132932,222.74
3,1303132933,222.20
4,1303132934,222.11
